In [11]:
import base64
from PIL import Image
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from vertexai.generative_models._generative_models import HarmCategory, HarmBlockThreshold, ResponseBlockedError
from pathlib import Path

vertexai.init(project="crypto-resolver-346012", location="us-central1")

def generate(image):
    model = GenerativeModel("gemini-pro-vision")
    
    safety_settings = {
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    }

    # Yes,But memes are funny because the Yes picture depicts a normal situation, and the But picture reveals something about the Yes picture that makes humans laugh. What is funny about the given Yes,But meme
    
    responses = model.generate_content(
        [image, """This meme made me laugh. What is the punchline in this meme?"""],
        generation_config={
            "max_output_tokens": 128,
            "temperature": 0.8,
            "top_p": 1,
            "top_k": 32,
            "candidate_count": 1
        },
        safety_settings=safety_settings,
        
    )
    #print(type(responses))
    #print(responses)
    #print(responses.text)
    return responses.text


with open("yesbut_second_round/20240101_172315_3D_YES_3D_BUT.jpg", "rb") as f:
    data = f.read()
    image1 = Part.from_data(data=data, mime_type="image/jpeg")
    #print(image1)

generate(image1)

" The punchline in this meme is that the woman's feet are extremely swollen and injured from wearing high heels."

In [14]:
import os
def to_dalle_path(type, og_filename, side):
    return os.path.join(f"outputs/dalle3/{'3d-black-silhouettes' if type=='3D' else 'original'}/{side}",f"{og_filename}_{side.upper()}_gen_1.jpg")

def get_lr_path_from_filename(filename):
    if not "YES" in filename and not "BUT" in filename:
        yes = os.path.join("images_split",f"{filename[:-4]}_YES.jpg")
        but = os.path.join("images_split",f"{filename[:-4]}_BUT.jpg")
    else:
        parts = filename.split("_")
        #print(parts)
        og_filename = "_".join(parts[:-4])
        yestype, buttype = parts[-4], parts[-2]
        yes = to_dalle_path(yestype, og_filename, "yes") if yestype!="ORIG" else os.path.join("images_split",f"{og_filename}_YES.jpg")
        but = to_dalle_path(buttype, og_filename, "but") if buttype!="ORIG" else os.path.join("images_split",f"{og_filename}_BUT.jpg")
    return yes,but

print(get_lr_path_from_filename("20240101_172315_STICK_YES_STICK_BUT.jpg"))

('outputs/dalle3/original/yes/20240101_172315_YES_gen_1.jpg', 'outputs/dalle3/original/but/20240101_172315_BUT_gen_1.jpg')


In [13]:
import os,json
from tqdm import tqdm
from validate import validate_file

outpath = "outputs/second_round/right/gemini-pro-vision-right.json"
inpath = "yesbut_second_round"
try:
    with open(outpath, "r") as f:
        outputs = json.load(f)
except Exception:
    print("starting from zero")
    outputs = {}

# files = [x for x in os.listdir(inpath) if "BUT" in x]
#redo_files = ["20240101_174949_BUT.jpg"]
files = []
fullfiles = os.listdir(inpath)
for filename in fullfiles:
    yes,but = get_lr_path_from_filename(filename)
    files.append(yes)

#for filename in tqdm(files):
for filepath in tqdm(files):
    filename = filepath.split("/")[-1]
    inpath = "/".join(filepath.split("/")[:-1])
    if filename in outputs and outputs[filename]:
        continue
    with open(os.path.join(inpath,filename), "rb") as f:
        data = f.read()
        image = Part.from_data(data=data, mime_type="image/jpeg")
    #display(Image.open(os.path.join("images",filename)).convert('RGB'))
    #print(filename)
    try:
        output = generate(image)
    except Exception as e:
        print("Caught exception: ",str(e))
        print("Could not do file: ",filename)
        output = ""
    
    outputs[filename]= output
    with open(outpath, "w") as f:
        json.dump(outputs, f, indent=4)

with open(outpath, "w") as f:
    json.dump(outputs, f, indent=4)
if not validate_file(outpath):
    print("Validation failed!")

starting from zero


  9%|█████████████                                                                                                                                          | 69/801 [07:14<1:01:47,  5.07s/it]

Caught exception:  list index out of range
Could not do file:  20240101_173813_3D_YES_STICK_BUT.jpg


 25%|██████████████████████████████████████▎                                                                                                                 | 202/801 [21:30<49:53,  5.00s/it]

Caught exception:  list index out of range
Could not do file:  20240101_180247_STICK_YES_ORIG_BUT.jpg


...

Caught exception:  list index out of range
Could not do file:  20240101_175031_ORIG_YES_3D_BUT.jpg


 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████            | 737/801 [1:17:05<05:08,  4.82s/it]

Caught exception:  list index out of range
Could not do file:  20240101_174949_ORIG_YES_STICK_BUT.jpg


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [1:23:45<00:00,  6.27s/it]

File outputs/second_round/punchline/gemini-pro-vision-punchline.json is valid.


In [ ]:
"""
Caught exception:  500 Internal error encountered.
Could not do file:  20240101_172932.jpg

Caught exception:  500 Internal error encountered.
Could not do file:  20240101_182249.jpg
"""

In [ ]:
"""
manually redid:
["20240101_173205_BUT.jpg", "20240101_173205_YES.jpg", "20240101_174949_BUT.jpg", "20240101_174949_YES.jpg", "20240101_181310_YES.jpg"]
"""